In [99]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fairbet_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [100]:
from order_placed.models import *
from django_pandas.io import read_frame
from django.db.models import F, Q, When, Value, Case, Sum

In [101]:
df = read_frame(Betting.objects.filter(match="Srilanka vs India"))

In [102]:
df

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,7,fairbet,1000.0,Srilanka,BACK,2.85,Srilanka,1850.0,Srilanka vs India,2022-07-04 07:05:52.327891+00:00,2022-07-05 06:22:08.226863+00:00
1,8,fairbet,1000.0,Srilanka,LAY,1.85,Srilanka,-850.0,Srilanka vs India,2022-07-04 07:06:03.985142+00:00,2022-07-05 06:16:39.476528+00:00
2,9,fairbet,1000.0,India,BACK,2.85,Srilanka,-1000.0,Srilanka vs India,2022-07-04 07:07:32.928877+00:00,2022-07-05 06:16:43.497817+00:00
3,10,fairbet,500.0,India,LAY,1.65,Srilanka,500.0,Srilanka vs India,2022-07-04 07:08:36.692581+00:00,2022-07-05 06:16:46.587269+00:00


In [40]:
# Client.objects.update(account_type=Case(
#     When(registered_on__lte=a_year_ago,
#          then=Value(Client.PLATINUM)),
#     When(registered_on__lte=a_month_ago,
#          then=Value(Client.GOLD)),
#     default=Value(Client.REGULAR)
# ),)

In [41]:
# betting = Betting.objects.filter(match="Srilanka vs India").values('loss_profit')

In [42]:
# bet = betting.annotate(
#     betting_loss_profit=Case(
#         When(
#             status="BACK",
#             then=F("amount")*0.8,
#         ),
#         default = F("loss_profit"),
#         output_field = models.FloatField()
#     )
# )

In [43]:
# Betting.objects.update(
#     loss_profit=Case(
#         When(status="B",
#              match="Srilanka vs India",
#              winning_team = "Not declared yet",
#              then=F("amount")*F("odds") - F("amount")),
#         When(status="L",
#              match="Srilanka vs India",
#              winning_team = "Not declared yet",
#              then=F("amount")* -F("odds") + F("amount")),
#         default=F("loss_profit")
# )
#      )

In [145]:
winningTeam = "India"
matchName = "Srilanka vs India"

In [148]:
betting_instance = Betting.objects.filter(
        match = matchName
    ).update(
        loss_profit = Case(
            When(
                status = "B",
                bet_on_team = winningTeam,
                winning_team = winningTeam,
                then = F('amount') * F("odds")),
            When(
                ~Q(bet_on_team = winningTeam),
                status = "B",
                winning_team = winningTeam,
                then = -F('amount')),
             When(
                status = "L",
                bet_on_team = winningTeam,
                winning_team = winningTeam,
                then = F('amount') * - F("odds")),
             When(
                 ~Q(bet_on_team = winningTeam),
                status = "L",
                winning_team = winningTeam,
                then = F('amount')),
            default=F("loss_profit")
            )
        )

In [103]:
from order_placed.models import *
from django_pandas.io import read_frame
from django.db.models import F, Q, When, Value, Case, Sum

In [131]:
bettings = Betting.objects.filter(loss_profit__gt=0)

In [132]:
bettings

<BulkUpdateOrCreateQuerySet [<Betting: Betting object (7)>, <Betting: Betting object (10)>]>